# 2025 DL Lab3: Semi-Supervised Flower Classfication

Before we start, please put **your name** and **SID** in following format: <br>
Hi I'm 陸仁賈, 314831000.

**Your Answer:**    
Hi I'm 戴良育, 314834012.

## Semi-supervised Flower Classfication

In this approach, you have a dataset that includes both labeled and unlabeled examples.

The goal is to use the labeled data to train the model while also leveraging the unlabeled
data to improve the model's performance.

In this assignment, you’ll explore a self-training mechanism for this task.


**Please note that you’re not allowed to use pre-constructed models or pre-trained weights.**

## Kaggle Competition
Kaggle is an online community of data scientists and machine learning practitioners. Kaggle allows users to find and publish datasets, explore and build models in a web-based data-science environment, work with other data scientists and machine learning engineers, and enter competitions to solve data science challenges.

This assignment use kaggle to calculate your grade.  
Please use this [**LINK**](https://www.kaggle.com/t/a611e0096e5943cc99a1c0545be28c3c) to join the competition.


##  Versions of used packages

We will check PyTorch version to make sure everything work properly.

In [1]:
import sys
import torch
import torchvision
print('python', sys.version.split('\n')[0])
print('torch', torch.__version__)
print('torchvision', torchvision.__version__)

python 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 16:37:03) [MSC v.1929 64 bit (AMD64)]
torch 2.8.0+cu126
torchvision 0.23.0+cu126


# Prepare Data

We use [Flowers Recognition](https://www.kaggle.com/alxmamaev/flowers-recognition) dataset.
This is collected by Alexander Mamaev.

**Abstrct**  

We clean the dataset,this dataset contains 4262 flower images.   
**IMPORTANT: you CANNOT use any extra images.**

The data collection is grabed from the data flicr, google images, yandex images.
You can use this datastet to recognize plants from the photo.

The pictures are divided into five classes:
+ daisy
+ tulip
+ rose
+ sunflower
+ dandelion

For each class there are about 800 photos. Photos are not high resolution, about 320x240 pixels. Photos are not reduced to a single size, they have different proportions!

## Unzip Data

Unzip `Lab3_data_flower_2025.zip`, there are 3 folders.

- `train/`: 6 subfolders total.
   - `daisy/`, `dandelion/`, `rose/`, `sunflower/`, `tulip/`: labeled training images.
   - `unlabel/`: unlabeled training images.
contains 6 folders for 5 categories of flowers. Images of flowers inside them.
- `val/`: contains 5 folders for the same 5 classes. Labeled validation images for each class.
- `test/`: unclassified images of testing set.
---

There are **1200 images in labeled training set.**  

There are **1202 images in unlabeled training set.**

There are **678 images in validation set.**

There are **1215 images in test set.**  


## Loading the dataset

In [2]:
data_folder = 'Lab3_data_flower_2025'

### Custom dataset

Build a classs inherit `torch.utils.data.Dataset`.  
Implement `__init__`, `__getitem__` and `__len__` 3 functions.  

Some operations could be there: setting location of dataset, the method of reading data, label of dataset or transform of dataset.

See [torch.utils.data.Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) for more details

In [4]:
import csv
import os.path as osp
from pathlib import Path
from PIL import Image
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import pandas as pd

CLASS_NAMES = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
CLASS_TO_IDX = {c:i for i, c in enumerate(CLASS_NAMES)}
VALID_EXTS = {'.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.webp'}

class FlowerData(Dataset):
    def __init__(self, root, split='train', mode='train', transform=None, use_unlabel=False):
        self.root = Path(root)
        self.split = split
        self.mode = mode
        self.transform = transform
        self.use_unlabel = use_unlabel

        self.paths = []
        self.labels = []
        self.rel_paths = []

        # Load data from unified CSV files
        if split == 'train' and use_unlabel:
            csv_file = self.root / 'unlabeled_train.csv'
        elif split == 'train':
            csv_file = self.root / 'train.csv'
        elif split == 'val':
            csv_file = self.root / 'val.csv'
        else:  # test
            csv_file = self.root / 'test.csv'

        # Read CSV file using pandas for better handling
        df = pd.read_csv(csv_file)

        for _, row in df.iterrows():
            file_path = self.root / row['file_name']
            self.paths.append(file_path)
            self.rel_paths.append(row['file_name'])

            # Handle labels
            if split == 'test' or (split == 'train' and use_unlabel):
                # No labels for test or unlabeled data
                pass
            else:
                # For labeled data
                if pd.isna(row['groundtruth']) or row['groundtruth'] == '':
                    self.labels.append(-1)  # Invalid label for debugging
                else:
                    self.labels.append(CLASS_TO_IDX[row['groundtruth']])

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        img_path = self.paths[index]
        img = Image.open(img_path).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)

        if self.mode == 'test' or (self.split == 'train' and self.use_unlabel):
            return img
        label = int(self.labels[index])
        return img, torch.tensor(label, dtype=torch.long)

### Data augmentation

Data augmentation are techniques used to increase the amount of data by adding slightly modified copies of already existing data or newly created synthetic data from existing data.

PyTorch use `torchvision.transforms` to do data augmentation.
[You can see all function here.](https://docs.pytorch.org/vision/main/transforms.html)  

There are some operations may not be necessary for predict, so we should write one for train and one for others.  
**NOTICE**：Please use v2 instead of transform cause some of function in v1 will be removed in the following version pytorch.

In [5]:
from torchvision.transforms import v2 as transforms
# For TRAIN
########################################################################
#  TODO: use transforms.xxx method to do some data augmentation        #
#  This one is for training, find the composition to get better result #
########################################################################
transforms_train = transforms.Compose([
    # 隨機裁切與縮放，避免直接拉伸圖片
    transforms.RandomResizedCrop(
        size=224,
        scale=(0.6, 1.0),           # 放寬取樣比例，讓視角更豐富
        ratio=(0.75, 1.33),
        antialias=True
    ),
    transforms.RandomHorizontalFlip(p=0.5),

    # 自動增強：使用 TrivialAugmentWide 或 RandAugment（二者擇一）
    transforms.TrivialAugmentWide(),
    # 若版本不支援 TrivialAugmentWide，可用 RandAugment 代替：
    # transforms.RandAugment(num_ops=2, magnitude=7),

    # 色彩抖動：使用 RandomApply 包裹，避免每張都強度過高
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)
    ], p=0.5),

    # 少量轉灰階，提高對顏色變化的穩健度
    transforms.RandomGrayscale(p=0.05),

    # 轉成 Tensor 並做正規化
    transforms.ToImage(),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

    # 遮擋式正則：RandomErasing
    transforms.RandomErasing(p=0.25, scale=(0.02, 0.12), ratio=(0.3, 3.3))
])
########################################################################
#                           End of your code                           #
########################################################################

# For VAL, TEST
########################################################################
#  TODO: use transforms.xxx method to do some data augmentation        #
#  This one is for validate and test,                                  #
#  NOTICE some operation we usually not use in this part               #
########################################################################
transforms_test = transforms.Compose([
    transforms.Resize(256, antialias=True),  # 維持長寬比
    transforms.CenterCrop(224),
    transforms.ToImage(),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
########################################################################
#                           End of your code                           #
########################################################################

### Instantiate dataset

Let's instantiate three `FlowerData` class.
+ train_set: for labeled_training.
+ unlabeled_set: for unlabeled_training.
+ dataset_val: for validation.

In [8]:
train_set       = FlowerData(data_folder, split='train', mode='train', transform=transforms_train, use_unlabel=False)
unlabeled_set   = FlowerData(data_folder, split='train', mode='test',  transform=transforms_test,  use_unlabel=True)  # train/unlabel
valid_set       = FlowerData(data_folder, split='val',   mode='train', transform=transforms_test,  use_unlabel=False)

num_classes = len(CLASS_NAMES)
print("The first image's shape in dataset_train :", train_set[0][0].size())
print("There are", len(train_set), "images in labeled_dataset_train.")
print("There are", len(unlabeled_set), "images in unlabeled_dataset_train.")
print("There are", len(valid_set), "images in dataset_val.")

# Verify the new format by checking a few samples
print("\nVerifying data loading with new CSV format:")
print("Train set - first sample label:", train_set[0][1].item() if len(train_set) > 0 else "No data")
print("Train set - file path:", train_set.rel_paths[0] if len(train_set) > 0 else "No data")
print("Unlabeled set - file path:", unlabeled_set.rel_paths[0] if len(unlabeled_set) > 0 else "No data")
print("Val set - first sample label:", valid_set[0][1].item() if len(valid_set) > 0 else "No data")
print("Val set - file path:", valid_set.rel_paths[0] if len(valid_set) > 0 else "No data")

The first image's shape in dataset_train : torch.Size([3, 224, 224])
There are 1200 images in labeled_dataset_train.
There are 1202 images in unlabeled_dataset_train.
There are 678 images in dataset_val.

Verifying data loading with new CSV format:
Train set - first sample label: 0
Train set - file path: train/daisy/14167534527_781ceb1b7a_n.jpg
Unlabeled set - file path: train/unlabel/unlabel_51e2a07b21.jpg
Val set - first sample label: 0
Val set - file path: val/daisy/521762040_f26f2e08dd.jpg


### DataLoader

`torch.utils.data.DataLoader` define how to sample from `dataset` and some other function like:
+ shuffle : set to `True` to have the data reshuffled at every epoch
+ batch_size : how many samples per batch to load

See [torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) for more details

In [29]:
#####################################################
#            You can adjust batch_size              #
#####################################################
batch_size = 32
num_workers = 0
loader_kwargs = dict(batch_size=batch_size, num_workers=num_workers, pin_memory=torch.cuda.is_available())
# if num_workers > 0:
#     loader_kwargs["persistent_workers"] = True

train_loader = DataLoader(train_set, shuffle=True,drop_last=True, **loader_kwargs)
val_loader   = DataLoader(valid_set, shuffle=False, **loader_kwargs)

Finally! We have made all data prepared.  
Let's go develop our model.

# Self-training

## Step 1: Supervised training

### Implement CNN using PyTorch

Try to use labeled data design and train a deep convolutional network from scratch to predict the class label of a flower image.

**Again, the goal of this assignment is for you to test different convolutional structures. You cannot directly use the blocks/architectures of pre-trained models.**

In [11]:

import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    """
    ResNet BasicBlock
    """
    expansion = 1
    def __init__(self, in_c, out_c, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_c, out_c, 3, stride, 1, bias=False)
        self.bn1   = nn.BatchNorm2d(out_c)
        self.relu  = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_c, out_c, 3, 1, 1, bias=False)
        self.bn2   = nn.BatchNorm2d(out_c)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.downsample is not None:
            identity = self.downsample(x)
        out = self.relu(out + identity)
        return out


def make_layer(in_c, out_c, blocks, stride):
    """
    創建 ResNet layer
    """
    down = None
    if stride != 1 or in_c != out_c:
        down = nn.Sequential(
            nn.Conv2d(in_c, out_c, 1, stride, bias=False),
            nn.BatchNorm2d(out_c)
        )
    layers = [BasicBlock(in_c, out_c, stride, down)]
    for _ in range(1, blocks):
        layers.append(BasicBlock(out_c, out_c))
    return nn.Sequential(*layers)

class ResNet34(nn.Module):
    """
    ResNet34 架構
    Layer 結構：[3, 4, 6, 3] blocks
    """
    def __init__(self, num_classes=5):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
        )
        # ResNet34: [3, 4, 6, 3] blocks (vs ResNet18: [2, 2, 2, 2])
        self.layer1 = make_layer(64,   64,  blocks=3, stride=1)
        self.layer2 = make_layer(64,   128, blocks=4, stride=2)
        self.layer3 = make_layer(128,  256, blocks=6, stride=2)
        self.layer4 = make_layer(256,  512, blocks=3, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, num_classes)
        )

        # Kaiming 初始化
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d)):
                nn.init.constant_(m.weight, 1); nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01); nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.stem(x)
        x = self.layer1(x); x = self.layer2(x); x = self.layer3(x); x = self.layer4(x)
        x = self.avgpool(x)
        return self.head(x)


In [13]:
from torch.nn.modules.conv import Conv2d
import torch.nn as nn
import torch.nn.functional as F
class YourCNNModel(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        ########################################################################
        #     TODO: use nn.xxx method to generate a CNN model part             #
        ########################################################################
        self.model = ResNet34(num_classes=num_classes)
        ########################################################################
        #                           End of your code                           #
        ########################################################################

    def forward(self, x):
        assert isinstance(x, torch.Tensor), "Input should be a torch Tensor"
        assert x.dim() == 4, "Input should be NHWC format"
        ########################################################################
        #     TODO: forward your model and get output                          #
        ########################################################################
        out = self.model(x)
        ########################################################################
        #                           End of your code                           #
        ########################################################################
        return out

In [14]:
device = torch.device('cuda')
# or
# device = torch.device('cpu')

In [15]:
model = YourCNNModel(num_classes=num_classes)
model = model.to(device)

We have made our model!  
Next, PyTorch also provide many utility function(loss, optmizer...etc).  
You can define them in one-line.

### Define loss and optimizer

[Optimizers in pytorch](https://docs.pytorch.org/docs/stable/optim.html)  
[CrossEntropyLoss in pytorch](https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)

In [31]:
import torch.nn as nn
import torch.optim as optim
################################################################################
# TODO: Define loss and optmizer functions                                     #
# Try any loss or optimizer function and learning rate to get better result    #
# hint: torch.nn and torch.optim                                               #
################################################################################
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.003, weight_decay=1e-4)
################################################################################
#                               End of your code                               #
################################################################################
criterion = criterion.to(device)

### Train the model

#### Train function
Let's define train function.  
It will iterate input data 1 epoch and update model with optmizer.  
Finally, calculate mean loss and total accuracy.

Hint: [torch.max()](https://pytorch.org/docs/stable/generated/torch.max.html#torch-max) or [torch.argmax()](https://pytorch.org/docs/stable/generated/torch.argmax.html)

In [17]:
from tqdm import tqdm
import torch

def train(input_data, model, criterion, optimizer, epoch=None, total_epochs=None):
    '''
    Argement:
    input_data -- iterable data, typr torch.utils.data.Dataloader is prefer
    model -- nn.Module, model contain forward to predict output
    criterion -- loss function, used to evaluate goodness of model
    optimizer -- optmizer function, method for weight updating
    '''
    model.train()
    loss_list = []
    total_count = 0
    acc_count = 0

    desc = f"Train | epoch {epoch}/{total_epochs}" if epoch is not None else "Train"
    pbar = tqdm(input_data, desc=desc, leave=False)

    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)

        ########################################################################
        # TODO: Forward, backward and optimize                                 #
        # 1. zero the parameter gradients                                      #
        # 2. process input through the network                                 #
        # 3. compute the loss                                                  #
        # 4. propagate gradients back into the network's parameters            #
        # 5. Update the weights of the network                                 #
        ########################################################################
        optimizer.zero_grad()  # 1. 清空梯度
        outputs = model(images)  # 2. 前向傳播，通過網路處理輸入
        loss = criterion(outputs, labels)  # 3. 計算損失
        loss.backward()  # 4. 反向傳播，計算梯度
        optimizer.step()  # 5. 更新網路權重
        ########################################################################
        #                           End of your code                           #
        ########################################################################


        ########################################################################
        # TODO: Get the counts of correctly classified images                  #
        # 1. get the model predicted result                                    #
        # 2. sum the number of this batch predicted images                     #
        # 3. sum the number of correctly classified                            #
        # 4. save this batch's loss into loss_list                             #
        # dimension of outputs: [batch_size, number of classes]                #
        # Hint 1: use outputs.data to get no auto_grad                         #
        # Hint 2: use torch.max()                                              #
        ########################################################################
        _, predicted = torch.max(outputs.data, 1)  # 1. 獲取預測結果（最大值的索引）
        total_count += labels.size(0)  # 2. 累加此批次的圖片數量
        acc_count += (predicted == labels).sum().item()  # 3. 累加正確分類的數量
        loss_list.append(loss.item())  # 4. 將此批次的損失加入列表
        ########################################################################
        #                           End of your code                           #
        ########################################################################

        running_acc = acc_count / total_count if total_count else 0.0
        lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(loss=f"{loss.item():.4f}", acc=f"{running_acc:.4f}", lr=f"{lr:.6f}")

    acc  = acc_count / total_count if total_count else 0.0
    loss = sum(loss_list) / len(loss_list) if loss_list else 0.0
    return acc, loss

#### Validate function
Next part is validate function.  
It works as training function without optmizer and weight-updating part.

In [19]:
def val(input_data, model, criterion, epoch=None, total_epochs=None):
    model.eval()
    loss_list = []
    total_count = 0
    acc_count = 0

    desc = f"Val   | epoch {epoch}/{total_epochs}" if epoch is not None else "Val"
    pbar = tqdm(input_data, desc=desc, leave=False)

    with torch.no_grad():
        for images, labels in pbar:
            images = images.to(device)
            labels = labels.to(device)

            ####################################################################
            # TODO: Get the predicted result and loss                          #
            # 1. process input through the network                             #
            # 2. compute the loss                                              #
            # 3. get the model predicted result                                #
            # 4. get the counts of correctly classified images                 #
            # 5. save this batch's loss into loss_list                         #
            ####################################################################
            outputs = model(images)  # 1. 前向傳播
            loss = criterion(outputs, labels)  # 2. 計算損失
            _, predicted = torch.max(outputs.data, 1)  # 3. 獲取預測結果

            total_count += labels.size(0)  # 4. 累加圖片數量
            acc_count += (predicted == labels).sum().item()  # 4. 累加正確分類數量
            loss_list.append(loss.item())  # 5. 儲存損失
            ####################################################################
            #                         End of your code                         #
            ####################################################################

            running_acc = acc_count / total_count if total_count else 0.0
            pbar.set_postfix(loss=f"{loss.item():.4f}", acc=f"{running_acc:.4f}")

    acc  = acc_count / total_count if total_count else 0.0
    loss = sum(loss_list) / len(loss_list) if loss_list else 0.0
    return acc, loss

#### Training in a loop
Call train and test function in a loop.  
Take a break and wait.

In [13]:
################################################################################
#     You can adjust those hyper parameters to loop for max_epochs times       #
################################################################################
max_epochs = 200
log_interval = 3
# 學習率調度器：CosineAnnealingLR
# - T_max：學習率從初始值降到最小值所需的 epoch 數（設為總 epoch 數）
# - eta_min：學習率的最小值，默認為 0
# - 使用餘弦退火策略，學習率會平滑地從初始值降到最小值
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epochs, eta_min=0.0005)

train_acc_list = []
train_loss_list = []
val_acc_list = []
val_loss_list = []


# -------- Early stopping 參數 --------
patience = 12  # 如果驗證集連續 N 回合沒有改善就停止
best_val_loss = float('inf')
patience_counter = 0
best_model_state = None
# ------------------------------------


for epoch in range(1, max_epochs + 1):
    train_acc, train_loss = train(train_loader, model, criterion, optimizer, epoch=epoch, total_epochs=max_epochs)
    val_acc, val_loss     = val(val_loader, model, criterion, epoch=epoch, total_epochs=max_epochs)

    train_acc_list.append(train_acc)
    train_loss_list.append(train_loss)
    val_acc_list.append(val_acc)
    val_loss_list.append(val_loss)

    scheduler.step()

      # 紀錄最佳模型權重
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # 深拷貝一份模型權重
        best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        torch.save(model.state_dict(), 'supurvised.pt')
        print(f"[Best updated] Saved checkpoint: {'supurvised.pt'} (val_loss={best_val_loss:.6f})")
    else:
        patience_counter += 1
    # 每 log_interval 輸出一次
    if epoch % log_interval == 0:
        lr = optimizer.param_groups[0]['lr']
        print('=' * 20, f'Epoch {epoch}/{max_epochs}', '=' * 20)
        print('Train Acc: {:.6f} | Train Loss: {:.6f}'.format(train_acc, train_loss))
        print('  Val Acc: {:.6f} |   Val Loss: {:.6f} | LR: {:.6f}'.format(val_acc, val_loss, lr))

    # 判斷是否早停
    if patience_counter >= patience:
        print(f'Early stopping triggered at epoch {epoch} (val_loss 未改善 {patience} 次)')
        break


################################################################################
#                               End of your code                               #
################################################################################

[Best updated] Saved checkpoint: supurvised.pt (val_loss=1.873921)


[Best updated] Saved checkpoint: supurvised.pt (val_loss=1.347375)


==================== Epoch 3/200 ====================
Train Acc: 0.436655 | Train Loss: 1.404377
  Val Acc: 0.401180 |   Val Loss: 1.381492 | LR: 0.002999


[Best updated] Saved checkpoint: supurvised.pt (val_loss=1.298551)


==================== Epoch 6/200 ====================
Train Acc: 0.445946 | Train Loss: 1.365692
  Val Acc: 0.417404 |   Val Loss: 1.352812 | LR: 0.002994


...

[Best updated] Saved checkpoint: supurvised.pt (val_loss=0.696062)
==================== Epoch 81/200 ====================
Train Acc: 0.690034 | Train Loss: 0.784447
  Val Acc: 0.772861 |   Val Loss: 0.696062 | LR: 0.002118


==================== Epoch 84/200 ====================
Train Acc: 0.688345 | Train Loss: 0.830803
  Val Acc: 0.727139 |   Val Loss: 0.768872 | LR: 0.002061


==================== Epoch 87/200 ====================
Train Acc: 0.720439 | Train Loss: 0.758291
  Val Acc: 0.734513 |   Val Loss: 0.774151 | LR: 0.002003


==================== Epoch 90/200 ====================
Train Acc: 0.715372 | Train Loss: 0.761606
  Val Acc: 0.705015 |   Val Loss: 0.743461 | LR: 0.001946


==================== Epoch 93/200 ====================
Train Acc: 0.718750 | Train Loss: 0.742075
  Val Acc: 0.740413 |   Val Loss: 0.707693 | LR: 0.001887
Early stopping triggered at epoch 93 (val_loss 未改善 12 次)


In [32]:
SUPERVISED_CKPT = 'supurvised.pt'
# torch.save(model.state_dict(), SUPERVISED_CKPT)

#### Visualize accuracy and loss

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 4))
# plt.plot(range(len(train_loss_list)), train_loss_list)
# plt.plot(range(len(val_loss_list)), val_loss_list)
# plt.legend(['train', 'val'])
# plt.title('Loss')
# plt.show()

# plt.figure(figsize=(12, 4))
# plt.plot(range(len(train_acc_list)), train_acc_list)
# plt.plot(range(len(val_acc_list)), val_acc_list)
# plt.legend(['train', 'val'])
# plt.title('Acc')
# plt.show()

: 

finish training your classifier, next you should use this classifer to predict unlabel images with pseduo label.

## Step2: Use unlabeled data to enhance model performance

In [33]:
# load the trained classifier weights
ckpt = torch.load(SUPERVISED_CKPT, map_location=device)
model.load_state_dict(ckpt)

<All keys matched successfully>

In [34]:
# create a unlabeled data set list, we will use it later
unlabeled_set_list = []
for img in unlabeled_set:
    unlabeled_set_list.append(img)

print("Unlabeled pool size:", len(unlabeled_set_list))

Unlabeled pool size: 1202


### Use the trained classifier to generates pseudo-labels of a dataset.

In [36]:
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import ConcatDataset
###########################################################
#   You can adjust the threshold to get better result !   #
###########################################################
def get_pseudo_labels(model, threshold=0.85):

    global unlabeled_set_list
    model.eval()
    imgs_keep = []
    labels_keep = []
    remove_index = []
    soft_max = nn.Softmax(dim=1)
    with torch.no_grad():
        for idx, img in enumerate(tqdm(unlabeled_set_list, desc="Pseudo-labeling", leave=False)):
            img = img.to(device)
            img = img.unsqueeze(0)  # Add batch dimension
            #####################################################################################
            #     TODO:                                                                         #
            #     1. Foward the data, Using torch.no_grad() accelerates the forward process     #
            #     2. obtain the probability distributions by applying softmax on logits         #
            #     3. Filter the data with threshold                                             #
            #     4. Combine the labeled training data with the pseudo-labeled data             #
            #        to construct a new training set. then removed                              #
            #     5. the unlabeled data from unlabeled_set_list                                 #
            #     hint: ConcatDataset                                                           #
            #####################################################################################
            # 1. 前向傳播（已在 torch.no_grad() 中，加速推論過程）
            outputs = model(img)
            # 2. 使用 softmax 獲取機率分布
            probs = soft_max(outputs)

            # 獲取最大機率和預測類別
            max_prob, pred_label = torch.max(probs, 1)

            # 3. 根據閾值篩選資料
            if max_prob.item() >= threshold:
                # 4. 將符合閾值的圖片和標籤加入保留列表
                imgs_keep.append(img.squeeze(0).cpu())  # 移除 batch 維度並移到 CPU
                labels_keep.append(pred_label.item())
                # 5. 記錄要從未標記資料中移除的索引
                remove_index.append(idx)
            #####################################################################################
            #                           End of your code                                        #
            #####################################################################################

    # Remove processed images from unlabeled list
    for i in reversed(remove_index):
        del unlabeled_set_list[i]

    if imgs_keep:
        # Stack on CPU first, then create TensorDataset
        pseudo_imgs   = torch.stack(imgs_keep, dim=0)                      # [N, C, H, W] on CPU
        pseudo_labels = torch.tensor(labels_keep, dtype=torch.long)        # [N] on CPU
        pseudo_dataset = TensorDataset(pseudo_imgs, pseudo_labels)
        taken = len(pseudo_dataset)
    else:
        pseudo_dataset = TensorDataset(torch.empty(0,3,224,224), torch.empty(0,dtype=torch.long))
        taken = 0

    print(f"Labeled {taken} images this round. Remaining unlabeled: {len(unlabeled_set_list)}")

    # Clear GPU cache to free memory
    torch.cuda.empty_cache()

    return pseudo_dataset, taken

## Redeifine your optimizer if you want

In [37]:
################################################################################
# TODO: Define loss and optmizer functions                                     #
# Try any loss or optimizer function and learning rate to get better result    #
# hint: torch.nn and torch.optim                                               #
################################################################################
optimizer = optim.AdamW(model.parameters(), lr=0.0015, weight_decay=5e-3)
criterion = nn.CrossEntropyLoss(label_smoothing=0.2)
################################################################################
#                               End of your code                               #
################################################################################
criterion = criterion.to(device)

### Train the model

Let's define train function.  

Use the **get_pseudo_labels** function to get the new training set, then construct a new data loader for training.

It will iterate input data 1 epoch and update model with optmizer.  

Finally, calculate mean loss and total accuracy.

In [38]:
import sys
sys.setrecursionlimit(1000000)

#########################################################################################################
#         You can adjust those hyper parameters like epochs or threshold for training                   #
#########################################################################################################
n_epochs = 100
N = 1  # Update synthesis dataset every N epochs
log_interval =3  # Log every N epochs like supervised training
best_acc = 0.0
SELF_TRAIN_CKPT = 'self_training.pt'


patience_ssl = 12                    # 連續多少個 epoch 沒改善就停
best_val_loss_ssl = float('inf')    # 目前最佳驗證損失
patience_counter_ssl = 0            # 沒改善的累計次數

# 學習率調度器：CosineAnnealingLR (用於自我訓練階段)
# - T_max：設為自我訓練的總 epoch 數
# - eta_min：最小學習率設為初始學習率的 1/10
scheduler_ssl = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs, eta_min=0.0003)


# Initialize with original labeled data
current_train_dataset = train_set
train_loader_ssl = train_loader  # Initialize with original train loader
all_pseudo_datasets = []  # Store all pseudo datasets to accumulate them

for epoch in range(n_epochs):
    #########################################################################################################
    #    TODO:                                                                                              #
    #    In each epoch, relabel the unlabeled dataset for semi-supervised learning.                         #
    #    1. Obtain pseudo-labels for unlabeled data using trained model.(use get_pseudo_labels function)    #
    #    2. Construct a new dataset and a data loader for training.                                         #
    #    You can try different way to use the get_pseudo_label function maybe will get the better result.   #                                  #
    #########################################################################################################
     # 每 N 個 epoch 更新一次偽標籤
    if (epoch + 1) % N == 0:
        pseudo_dataset, taken = get_pseudo_labels(model, threshold=0.9)

        if taken > 0:
          all_pseudo_datasets.append(pseudo_dataset)

      # 合併數據集
        if all_pseudo_datasets:
            current_train_dataset = ConcatDataset([train_set] + all_pseudo_datasets)
        else:
            current_train_dataset = train_set

        # 重建數據加載器
        train_loader_ssl = DataLoader(
            current_train_dataset,
            shuffle=True,
            drop_last=True,
            **loader_kwargs
        )
    #########################################################################################################
    #                                          End of your code                                             #
    #########################################################################################################

    try:
        # ---------- Training using the train function from above ----------
        train_acc, train_loss = train(train_loader_ssl, model, criterion, optimizer, epoch=epoch+1, total_epochs=n_epochs)

        # ---------- Validation ----------
        valid_acc, valid_loss = val(val_loader, model, criterion, epoch=epoch+1, total_epochs=n_epochs)

                # >>> [Early Stopping - SSL]：以驗證損失作為是否「改善」的判準 <<<
        if valid_loss < best_val_loss_ssl:
            best_val_loss_ssl = valid_loss
            patience_counter_ssl = 0
            # 也一併更新對應的 acc 記錄（僅供觀察，不作為 early stopping 判斷）
            best_acc = max(best_acc, valid_acc)
            torch.save(model.state_dict(), SELF_TRAIN_CKPT)  # 保存目前最佳權重
        else:
            patience_counter_ssl += 1
        # <<< [Early Stopping - SSL] 結束 >>>

        # 學習率調整
        scheduler_ssl.step()

        # 記錄/輸出
        if (epoch + 1) % log_interval == 0:
            lr = optimizer.param_groups[0]['lr']
            total_pseudo = sum(len(dataset) for dataset in all_pseudo_datasets)
            print('=' * 20, f'Epoch {epoch+1}/{n_epochs}', '=' * 20)
            print('Train Acc: {:.6f} | Train Loss: {:.6f}'.format(train_acc, train_loss))
            print('  Val Acc: {:.6f} |   Val Loss: {:.6f} | LR: {:.6f}'.format(valid_acc, valid_loss, lr))
            print('Dataset Size: {} (original: {}, pseudo: {})'.format(len(current_train_dataset), len(train_set), total_pseudo))

        # >>> [Early Stopping - SSL]：達到耐心上限則提前停止 <<<
        if patience_counter_ssl >= patience_ssl:
            print(f'[SSL] Early stopping at epoch {epoch+1}: val_loss 未改善 {patience_ssl} 次，最佳 val_loss={best_val_loss_ssl:.6f}')
            break
        # <<< [Early Stopping - SSL] 結束 >>>

    except RuntimeError as e:
        print(f"CUDA error during training epoch {epoch+1}: {e}")
        torch.cuda.empty_cache()

        # Optionally reduce batch size if memory error
        if "out of memory" in str(e).lower():
            new_batch_size = max(batch_size // 2, 1)
            loader_kwargs_reduced = loader_kwargs.copy()
            loader_kwargs_reduced['batch_size'] = new_batch_size
            loader_kwargs_reduced['drop_last'] = True   # <— 補這行

            train_loader_ssl = DataLoader(
                current_train_dataset,
                shuffle=True,
                **loader_kwargs_reduced
            )

#########################################################################################################
#                               End of your code                                                        #
#########################################################################################################

Labeled 392 images this round. Remaining unlabeled: 810


Labeled 1 images this round. Remaining unlabeled: 809


Labeled 11 images this round. Remaining unlabeled: 798


==================== Epoch 3/100 ====================
Train Acc: 0.791250 | Train Loss: 1.018218
  Val Acc: 0.737463 |   Val Loss: 1.084716 | LR: 0.001497
Dataset Size: 1604 (original: 1200, pseudo: 404)


Labeled 2 images this round. Remaining unlabeled: 796


...

Labeled 0 images this round. Remaining unlabeled: 631


Labeled 0 images this round. Remaining unlabeled: 631


==================== Epoch 78/100 ====================
Train Acc: 0.912500 | Train Loss: 0.809438
  Val Acc: 0.833333 |   Val Loss: 0.943539 | LR: 0.000438
Dataset Size: 1771 (original: 1200, pseudo: 571)


Labeled 0 images this round. Remaining unlabeled: 631


[SSL] Early stopping at epoch 79: val_loss 未改善 12 次，最佳 val_loss=0.935457


### Predict Result

Predict the labesl based on testing set. Upload to [Kaggle](https://www.kaggle.com/t/a611e0096e5943cc99a1c0545be28c3c).

**How to upload**

1. Click the folder icon in the left hand side of Colab.
2. Right click "result.csv". Select "Download"
3. To kaggle. Click "Submit Predictions"
4. Upload the result.csv
5. System will automaticlaly calculate the accuracy of 50% dataset and publish this result to leaderboard.

In [39]:
# if you wanna load previous best model
ckpt = torch.load('self_training.pt', map_location=device)
model.load_state_dict(ckpt)

<All keys matched successfully>

In [40]:
test_set = FlowerData(data_folder, split='test', mode='test', transform=transforms_test)
test_loader = DataLoader(
    test_set,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=False,
    pin_memory=torch.cuda.is_available()
)

In [41]:
def predict(input_data, model):
    model.eval()
    output_list = []
    with torch.no_grad():
        for images in input_data:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            output_list.extend(predicted.to('cpu').numpy().tolist())
    return output_list

In [42]:
pred_indices = predict(test_loader, model)

with open('result.csv', 'w', newline='') as csvFile:
    writer = csv.DictWriter(csvFile, fieldnames=['ID', 'label'])
    writer.writeheader()
    for filename, pred in zip(test_set.paths, pred_indices):
        filename = osp.basename(filename)  # Extract just the filename
        writer.writerow({'ID': filename, 'label': CLASS_NAMES[int(pred)]})

print("Saved predictions to result.csv")

Saved predictions to result.csv
